In [22]:
from scipy.stats import binom # For question 3

# Constants
total_pairs = 162
p_not_show = 0.1
p_show = 0.9
max_not_showing_pairs = 12  # Calculated as pairs corresponding to 24 individuals

# Probability calculation using binomial distribution
probability_overbooked = sum(binom.pmf(k, total_pairs, p_not_show) for k in range(max_not_showing_pairs + 1))

probability_overbooked


0.16643753582955767

In [27]:
# 4.7
import random as rnd

def bernoulli(p):
    return 1 if rnd.random() <= p else 0

numberSamples = 1000000

P_A = 0.10  # Probability of being in Group A
P_B = 0.90  # Probability of being in Group B
P_C_given_A = 0.50  # Correct identification for Group A
P_C_given_B = 0.99  # Correct identification for Group B

numCorrect = 0
numMisidentified_A = 0
numMisidentifications = 0

for k in range(numberSamples):
    if bernoulli(P_A):
        group = 'A'
        correct = bernoulli(P_C_given_A)
    else:
        group = 'B'
        correct = bernoulli(P_C_given_B)
    
    if correct:
        numCorrect += 1
    else:
        numMisidentifications += 1
        if group == 'A':
            numMisidentified_A += 1

print(f"Out of {numberSamples:,} samples:")
print(f"Correct identifications: {numCorrect:,} ({numCorrect/numberSamples:.2%})")
print(f"Misidentifications: {numMisidentifications:,} ({numMisidentifications/numberSamples:.2%})")
print(f"Misidentifications from Group A: {numMisidentified_A:,} ({numMisidentified_A/numMisidentifications:.2%})")


Out of 1,000,000 samples:
Correct identifications: 941,211 (94.12%)
Misidentifications: 58,789 (5.88%)
Misidentifications from Group A: 49,561 (84.30%)


In [16]:
# Problem 5.1

import math
import scipy.integrate as integrate
from scipy.stats import norm
from IPython.display import Markdown, display
from tabulate import tabulate

# Define the Bernoulli function as provided
def bernoulli(p):
    x = rnd.random()
    if x <= p:
        return 1
    else:
        return 0

# Function to calculate exact binomial probability P(X >= k)
def exact_probability(n, p, k):
    return sum(binom.pmf(x, n, p) for x in range(k, n+1)) # this is probablity mass function 

# Function without continuity correction
def normal_approximation(n, p, k):
    mu = n * p
    sigma = math.sqrt(n * p * (1 - p))
    z = (k - mu) / sigma
    return 1 - norm.cdf(z)

# Function with continuity correction
def normal_approximation_cc(n, p, k):
    mu = n * p
    sigma = math.sqrt(n * p * (1 - p))
    # Apply continuity correction: subtract 0.5 for P(X >= k)
    z = (k - 0.5 - mu) / sigma
    return 1 - norm.cdf(z)

# Define the scenarios for each problem

rolls = [10, 100, 1000]

# creating a list of dictionaires

scenarios = [
    {
        "problem": "1. Rolling an even number more than 70% of the time (6-sided die)",
        "p": 0.5,
        "threshold_percent": 0.7
    },
    {
        "problem": "2. Rolling a 1 more than 20% of the time (6-sided die)",
        "p": 1/6,
        "threshold_percent": 0.2
    },
    {
        "problem": "3. Rolling a 1 more than 9% of the time (20-sided die)",
        "p": 1/20,
        "threshold_percent": 0.09
    }
]

# Function to process each scenario and generate tables
def process_scenarios(scenarios):
    for scenario in scenarios:
        problem = scenario["problem"]
        p = scenario["p"]
        roll_counts = rolls
        threshold = scenario["threshold_percent"]
        
        table = []
        headers = ["Number of Rolls (n)", "Threshold (k)", "Exact Probability", 
                   "Normal Approx. (No CC)", "Normal Approx. (With CC)"]
        
        for n in roll_counts:
            # Calculate threshold k = floor(a*n) + 1
            a_n = threshold * n
            k = math.floor(a_n) + 1 
            
            # Exact probability
            exact_prob = exact_probability(n, p, k)
            
            # Normal approximation without continuity correction
            normal_no_cc = normal_approximation(n, p, k)
            
            # Normal approximation with continuity correction
            normal_cc = normal_approximation_cc(n, p, k)
            
            # Append the results to the table
            table.append([
                n,
                k,
                f"{exact_prob:.6f}",
                f"{normal_no_cc:.6f}",
                f"{normal_cc:.6f}"
            ])
        
        # Display the table using tabulate and Markdown
        markdown_table = tabulate(table, headers=headers, tablefmt="github")
        display(Markdown(f"### {problem}\n\n{markdown_table}\n"))
        
# Process and display all scenarios
process_scenarios(scenarios)



# Discussion:

'''
Discuss any trends you see. 

    As the number of rolls increases, the probability of getting more than X% of that number decreases. 
    This decrease seems to follow a non-linear pattern.

How does the approximation behave as n increases or as thje probablities of the events decrease? 
    The approximation seems to get more accurate as n increases, the approximation also seems to get more accurate as the 
    probability of the event approaches 0.5 - bc yk binomial distribution is symmetric around 0.5

How does the  region you are integrating the normal curve over change as n increase? 
    The region seems to get smaller as n increases

Does this have any implications for the approximation by a normal?
    The normal approximation seems to get more accurate as n increases
'''


### 1. Rolling an even number more than 70% of the time (6-sided die)

|   Number of Rolls (n) |   Threshold (k) |   Exact Probability |   Normal Approx. (No CC) |   Normal Approx. (With CC) |
|-----------------------|-----------------|---------------------|--------------------------|----------------------------|
|                    10 |               8 |            0.054688 |                  0.02889 |                   0.056923 |
|                   100 |              71 |            1.6e-05  |                  1.3e-05 |                   2.1e-05  |
|                  1000 |             701 |            0        |                  0       |                   0        |


### 2. Rolling a 1 more than 20% of the time (6-sided die)

|   Number of Rolls (n) |   Threshold (k) |   Exact Probability |   Normal Approx. (No CC) |   Normal Approx. (With CC) |
|-----------------------|-----------------|---------------------|--------------------------|----------------------------|
|                    10 |               3 |            0.224773 |                 0.12895  |                   0.23975  |
|                   100 |              21 |            0.151888 |                 0.122464 |                   0.151836 |
|                  1000 |             201 |            0.002488 |                 0.001788 |                   0.002047 |


### 3. Rolling a 1 more than 9% of the time (20-sided die)

|   Number of Rolls (n) |   Threshold (k) |   Exact Probability |   Normal Approx. (No CC) |   Normal Approx. (With CC) |
|-----------------------|-----------------|---------------------|--------------------------|----------------------------|
|                    10 |               1 |            0.401263 |                 0.23408  |                   0.5      |
|                   100 |              10 |            0.028188 |                 0.010891 |                   0.019474 |
|                  1000 |              91 |            0        |                 0        |                   0        |


'\nDiscuss any trends you see. It seems like as the number of rolls increases the proabablity of getting more than X% of that number decreases How does the approximation behave as n increases or as thje probablities of the events decrease? How does the \nregion you are integrating the normal curve over change as n increase? Does this have any implications for the approximation by a normal?\n'

In [17]:
import tpqm

# Define the function to integrate
def f(x):
    return math.cos(x) * math.sin(2 * x) + 1

# Define the rectangle R - graphed on desmos for that
x_min, x_max = 0, 8
y_min, y_max = 0, 2  
area_R = (x_max - x_min) * (y_max - y_min)  

# Exact value of I
I_exact = 8.6687  # As computed from symbolab

# Monte Carlo estimation function
def monte_carlo_estimate(n):
    count = 0
    for _ in range(n):
        x = rnd.uniform(x_min, x_max)
        y = rnd.uniform(y_min, y_max)
        if y <= f(x):
            count += 1
    return (count / n) * area_R

# Simulation parameters
n = 5000
delta = 0.19  # As determined in Question 2
m = 10000  # Number of simulations

# Perform m simulations
success = 0
for _ in tqdm(range(m), desc="Simulations"):
    I_hat = monte_carlo_estimate(n)
    if abs(I_hat - I_exact) <= delta:
        success += 1

# Calculate empirical probability
empirical_probability = success / m
print(f"\nEmpirical Probability that |I_hat - I| <= {delta}: {empirical_probability:.4f}")


Simulations: 100%|██████████| 10000/10000 [00:21<00:00, 465.20it/s]


Empirical Probability that |I_hat - I| <= 0.19: 0.9074
